
    
    
## <H2 align="center"> <font color='white'> **MAF1731B**: Microstructure and Trading Systems </H2>

### <H3 align="center"> <font color='white'> **Laboratory 4**: Market Microstructure </H3>

 <H5 align="center"> <em> Professor: Villalobos Ramos Omar Antonio <em>

 <p align="center"> <img style=" margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="350" height="150" /> 

 <strong> <H5 align="left"> Student:
Esteban Márquez Delgado <strong>

 *Registry*:
if700637: https://github.com/EstebanMqz/MyST_LAB_4

 <center> <font color= #555555> <font size = 4> November 2022 </a> <font color= #555555> <font size = 4> </a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Laboratory 4 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure  </font>

<hr style="border:0.02in solid gray"> </hr>

### <font color= #555555> <font size = 4> Abstract </font>

This document was prepared as delivery for Laboratory 4 of Microstructure and Trading Systems - MAF1731B class during the Autumn 2022 course at ITESO for the Bachelor of Financial Engineering. In order to analyze the market microstructure in criptocurrencies markets containing the following 3 sections:

1. **Consume CCXT data**: Download Criptocurrencies Order Books from exchanges (kraken, ftx, currencycom, coinmate, etc).

2. **Microstructure Visualization**: Build Time Series using the Order Books obtained in the previous point from the following elements of the Market Microstructure for each asset and exchange (Levels, Bid Volume, Ask Volume, Total Volume, Mid Price, VWAP.)

3. **Microstructure Modeling**: Estimation of Roll Effective Spread using the lagged series at t-5.



<font color= lightblue> <font size = 4> 0. Libraries and Dependencies: </font>

<font color= gray>

**Note**: To run this notebook it is necessary to have the following libraries contained in the [requirements.txt](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/requirements.txt) script of this project.

To install the libraries in the script run the following cell:

In [34]:
!pip install -r requirements.txt

ERROR: Invalid requirement: '"""' (from line 1 of requirements.txt)

If you prefer, manual installation can be done with the individual installation of the following libraries:

<font color= lightblue> <font size = 1> 
+ pandas >= 1.3.4
+ numpy >= 1.19.1
+ jupyter >= 1.0.0
+ chart_studio >= 1.1
+ plotly >= 4.14
+ logging >= 3.11
+ datetime >= 4.7
+ pandas_datareader >= 0.10.0
+ tk >= 0.10
+ random >= 3.11
+ plotly >= 5.60
+ yahoofinancials >= 1.60
+ scipy >= 1.7.3
+ regex >= 2022.3.15
+ matplotlib >= 3.5.1
+ fire >= 0.4.0
+ ccxt >= 2.1.33
+ time >= 3.11 


### <font color= lightblue> <font size = 4> 0.1 Introduction: </font>
In this laboratory we will use information from the Cryptocurrency Order Book of various international exchanges, with the primary objective of visualizing and modeling the microstructure of the market through the behavior of the Order Book and its characteristics such as; Matching Engine, Levels, Ask-Bid Volumes, etc.
<font color= gray>




##### *General help:*
<font color= lightgreen> **Note**: Progress checks titles in sections highlighted. <font color= white>

In order, to retrieve specific help from the scripts tree in the project execute the help function on the script or in specific modules from the following:
+ [data.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/data.py)
+ [functions.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/functions.py)
+ [visualizations.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/visualizations.py)

In [35]:
## Libraries
import pandas as pd # Analysis and data management
import pandas_datareader as pdr
import pandas_datareader.data as web #Data extraction from the web
import numpy as np
from datetime import datetime, timedelta, date
from tkinter.ttk import Style
from tkinter import Y
import random as rand
import plotly.graph_objects as go #Plotly
import plotly.express as px
import warnings
from yahoofinancials import YahooFinancials # Yfinance 
import yfinance as yf
import ccxt #Criptocurrencies
import logging as log #Exchanges 
from scipy.optimize import minimize # Optimization 
from collections import Counter #Counter
import time
import re
import io 
import glob
import os
#.Py archives in repo
import functions as fn 
import visualizations as vs
import data as dt
from os import path
import fire
%matplotlib inline

<font color= orange> <font size = 5> 1. Consume CCXT data: </font>

In order to know the exchanges avaliable in CCTX module in Python for Criptocurrencies data download, the following getattr function *find_exchanges* is executed:

In [36]:
help(fn.find_exchanges)

Help on function find_exchanges in module functions:

find_exchanges(features=None, is_authenticated=False)
    Function that returns avaliable cryptocurrencies exchanges in Python for CCTX module.
    
        Parameters:
        ----------
        features: None 
        is_authenticated: None 
    
        Returns:
        -------
        exchange_names: Array of authenticated exchanges avaliable in CCTX.



From all the 120 exchanges avaliable we pick **binance**, **ftx** and **bytetrade** because of their fast fetching.

In [37]:
#Available exchanges in CCTX
fn.find_exchanges()[:] #len 120

['aax',
 'alpaca',
 'ascendex',
 'bequant',
 'bibox',
 'bigone',
 'binance',
 'binancecoinm',
 'binanceus',
 'binanceusdm',
 'bit2c',
 'bitbank',
 'bitbay',
 'bitbns',
 'bitcoincom',
 'bitfinex',
 'bitfinex2',
 'bitflyer',
 'bitforex',
 'bitget',
 'bithumb',
 'bitmart',
 'bitmex',
 'bitopro',
 'bitpanda',
 'bitrue',
 'bitso',
 'bitstamp',
 'bitstamp1',
 'bittrex',
 'bitvavo',
 'bkex',
 'bl3p',
 'blockchaincom',
 'btcalpha',
 'btcbox',
 'btcex',
 'btcmarkets',
 'btctradeua',
 'btcturk',
 'buda',
 'bw',
 'bybit',
 'bytetrade',
 'cex',
 'coinbase',
 'coinbaseprime',
 'coinbasepro',
 'coincheck',
 'coinex',
 'coinfalcon',
 'coinmate',
 'coinone',
 'coinspot',
 'crex24',
 'cryptocom',
 'currencycom',
 'delta',
 'deribit',
 'digifinex',
 'exmo',
 'flowbtc',
 'fmfwio',
 'ftx',
 'ftxus',
 'gate',
 'gateio',
 'gemini',
 'hitbtc',
 'hitbtc3',
 'hollaex',
 'huobi',
 'huobijp',
 'huobipro',
 'idex',
 'independentreserve',
 'indodax',
 'itbit',
 'kraken',
 'kucoin',
 'kucoinfutures',
 'kuna',
 'lat

Now that we have selected the exchanges, we pick random criptocurrencies and verify that they are all avaliable in their platform.

In [38]:
exchanges= ['binance','ftx', 'bytetrade'] #Selected exchanges from list (can't modify because of fn and vs .py)
criptos = ['BTC/USDT','ETH/USDT','XRP/USDT'] #Criptos OBs (modify tickers here)

<font color= lightblue> <font size = 4> Order Book Levels & OHLCV for 3 exchanges and Criptos: </font>

Now we proceed to download Single Order Books from exchanges for criptocurrencies.

In [39]:
help(vs.cctx_download)

Help on function cctx_download in module visualizations:

cctx_download(lvls, cripto, exchange)
    Function that returns prices and quantities of given levels (as integer) Bids & Asks in an Orderbook
    for the specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and
    from the following exchanges: binance, ftx or ascendex (as string).
    
        Parameters:
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable symbol (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        Returns:
        -------
        levels_ob_bid: Prices and Quantities of Bids (pos [0]) and Asks (pos [1]) as dataframes.



We get n=25 levels for each OB, and proceed to download criptos and exchanges from their lists [0:2] (9).

We will start with binance and BTC/USDT.

In [40]:
params= 25, criptos[0], exchanges[0]
params

(25, 'BTC/USDT', 'binance')

+ <font color= lightgray> <font size = 2> *Bids*: </font>

In [41]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,15930.29,0.05013
1,15929.57,0.02400
2,15929.44,0.01503
3,15929.11,0.00198
4,15929.09,0.17604


+ <font color= lightgray> <font size = 2> *Asks*: </font>

In [42]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,15931.16,0.00400
1,15931.17,1.02024
2,15931.44,0.04000
3,15931.45,0.06116
4,15933.07,0.09070


In [43]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T00:09:44.630Z,15798.68,15834.9,15762.86,15833.86,295.16243


We plot an **Order Book** for desired tickers and exchanges the visualization *OBLvls*.

In [44]:
help(vs.OBLvls_hist)

Help on function OBLvls_hist in module visualizations:

OBLvls_hist(lvls, cripto, exchange)
    Function that plots an horizontal histogram in plotly for CriptoCurrencies OB/
    
        Parameters
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable symbol (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        x: Quantity (col) of the given cripto (str) for given lvls (int) in Order Book.
        y: Prices (col) of the given cripto (str) for given lvls (int) in Order Book.
    
        Returns
        -------
        Histogram of Order Book prices and quantities for n given lvls.



In [45]:
vs.OBLvls_hist(params[0],params[1], params[2])

We will proceed for binance and ETH/USDT.

In [46]:
params= 25, criptos[1], exchanges[0]
params

(25, 'ETH/USDT', 'binance')

In [47]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,1096.97,1.1000
1,1096.95,15.0000
2,1096.94,0.0124
3,1096.93,0.0273
4,1096.61,1.2336


In [48]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,1096.98,7.0355
1,1097.01,2.0000
2,1097.06,0.4173
3,1097.07,8.6000
4,1097.10,3.0364


In [49]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T00:09:49.050Z,1095.03,1100.61,1091.76,1100.39,757.2307


In [50]:
vs.OBLvls_hist(params[0],params[1], params[2])

We continue with the exchange binance and XRP/USDT.

In [51]:
params= 25, criptos[2], exchanges[0]
params

(25, 'XRP/USDT', 'binance')

In [52]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,0.3342,14212.0
1,0.3341,22567.0
2,0.3340,37198.0
3,0.3339,28526.0
4,0.3338,48165.0


In [53]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,0.3343,11171.0
1,0.3344,3291.0
2,0.3345,122.0
3,0.3346,3756.0
4,0.3347,15134.0


In [54]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T00:09:52.440Z,0.3316,0.3318,0.3301,0.3318,212688.0


In [55]:
vs.OBLvls_hist(params[0],params[1], params[2])

We could retrieve criptocurrencies with any of the 3 exchanges declared in our list *exchanges* but **to avoid being repetitive we will make one more random example** and we will conclude with their dataframes in progress check. For section section **2.** (*Microstructure Visualization*) a dataframe with different columns and plots that allow us to compare differences between exchanges will be created.

In [56]:
params= 25, criptos[rand.randint(0,2)], exchanges[rand.randint(0,2)]
params

(25, 'ETH/USDT', 'binance')

In [57]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])

In [58]:
vs.OBLvls_hist(params[0],params[1], params[2])

In [59]:
v_params0= 25, criptos[0], exchanges[0]
v_params1= 25, criptos[0], exchanges[1]
v_params2= 25, criptos[0], exchanges[2]
v_params0, v_params1, v_params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

<strong> 
<font color= lightgreen> <font size = 3> Progress check 1 </font>

<font color= orange> <font size = 1> n = data fetch per exchange </font>

In [86]:
n=3 #Iterations 

In [60]:
help(vs.verif_ex1)

Help on function verif_ex1 in module visualizations:

verif_ex1(lvls, cripto, exchange, n)
    Function that returns Verif 1 for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable symbol (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    
    Returns:
    -------
    df1 = Section 1 Verification.
    df2 = OHLCV.



In [61]:
verif_0_B = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_B = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_B = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

Some differences between criptocurrencies within exchanges can be appreciated. Now we can realize which exchange from the sample can provide us with the lowest spread and best liquidity, and it is binance in this case. 

Clearly, prices are the most important factor for investments and that's why Mid-Price is going to be analyzed in the plots of section *2° (Microstructure Visualization)*. 

<font color= lightblue> <font size = 2> Verification BTC </font>

As it can be appreciated lower close prices of exchanges can be associated with bigger bid volumes mainly.

In [62]:
verif_df1 = fn.concatenate(verif_0_B[0], verif_1_B[0], verif_2_B[0], n)
verif_df1

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T00:10:00.015,15938.78,15937.58,0.85494,2.229970,1.20,15934.18
1,binance,2022-11-10T00:10:02.833,15927.30,15925.85,1.99285,5.152340,1.45,15926.50
2,binance,2022-11-10T00:10:05.278,15928.38,15926.09,1.75907,7.329270,2.29,15919.85
3,ftx,2022-11-10T00:10:08.212,15382.00,15369.00,10.01320,0.916400,13.00,15404.00
4,ftx,2022-11-10T00:10:11.192,15404.00,15370.00,10.01310,8.725500,34.00,15404.00
5,ftx,2022-11-10T00:10:15.191,15435.00,15371.00,11.19700,9.076700,64.00,15435.00
6,bytetrade,2022-11-10T00:10:17.782,16065.30,15800.50,3.56479,2.990453,264.80,15918.80
7,bytetrade,2022-11-10T00:10:20.441,16065.30,15800.50,3.56479,2.990453,264.80,15925.50
8,bytetrade,2022-11-10T00:10:22.884,16065.30,15800.50,3.56479,2.990453,264.80,15925.50


<font color= lightblue> <font size = 2> OHLCV BTC </font>

For each timeframe a candle is formed and it illustrates how much Volatility every timestamp shows within exchanges. This is important to know for algorithmic trading, not to mention the fact that stock exchanges do not provide candle data in *ms* for many reasons.

In [63]:
verif_df1_OH = fn.concatenate(verif_0_B[1], verif_1_B[1], verif_2_B[1], n)
verif_df1_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T00:10:00.015,15941.02,15952.05,15910.88,15934.18,685.55650
1,binance,2022-11-10T00:10:02.833,15926.07,15928.01,15920.20,15926.50,13.85285
2,binance,2022-11-10T00:10:05.278,15926.07,15928.01,15919.33,15919.85,20.39760
3,ftx,2022-11-10T00:10:08.212,15404.00,15404.00,15382.00,15404.00,1465.09720
4,ftx,2022-11-10T00:10:11.192,15404.00,15404.00,15360.00,15404.00,11481.55260
5,ftx,2022-11-10T00:10:15.191,15404.00,15436.00,15360.00,15435.00,20952.87230
6,bytetrade,2022-11-10T00:10:17.782,15927.80,15948.10,15918.80,15918.80,0.00983
7,bytetrade,2022-11-10T00:10:20.441,15927.80,15948.10,15918.80,15925.50,0.01053
8,bytetrade,2022-11-10T00:10:22.884,15927.80,15948.10,15918.80,15925.50,0.01053


<font color= lightblue> <font size = 2> Verification ETH </font>

In [64]:
v_params0= 25, criptos[1], exchanges[0]
v_params1= 25, criptos[1], exchanges[1]
v_params2= 25, criptos[1], exchanges[2]
v_params0, v_params1, v_params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

In [65]:
verif_0_E = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_E = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_E = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In [66]:
verif_df2 = fn.concatenate(verif_0_E[0], verif_1_E[0], verif_2_E[0], n)
verif_df2

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T00:10:25.841,1099.20,1099.19,61.3905,48.8613,0.01,1099.19
1,binance,2022-11-10T00:10:28.336,1099.01,1099.00,86.9299,46.3297,0.01,1099.00
2,binance,2022-11-10T00:10:30.798,1098.51,1098.50,48.9292,57.3924,0.01,1098.51
3,ftx,2022-11-10T00:10:35.308,1098.30,1097.80,3.4480,10.5480,0.50,1097.80
4,ftx,2022-11-10T00:10:39.814,1098.30,1098.20,0.3480,10.7810,0.10,1098.20
5,ftx,2022-11-10T00:10:44.889,1098.30,1098.20,0.3480,10.7800,0.10,1098.20
6,bytetrade,2022-11-10T00:10:48.192,1107.89,1095.34,15.0642,161.4832,12.55,1101.39
7,bytetrade,2022-11-10T00:10:50.673,1107.89,1095.34,15.0642,161.4832,12.55,1101.39
8,bytetrade,2022-11-10T00:10:54.556,1112.98,1095.34,18.4970,161.4225,17.64,1102.31


<font color= lightblue> <font size = 2> OHLCV ETH </font>

In [67]:
verif_df2_OH = fn.concatenate(verif_0_E[1], verif_1_E[1], verif_2_E[1], n)
verif_df2_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T00:10:25.841,1098.00,1101.05,1097.67,1099.19,330.8162
1,binance,2022-11-10T00:10:28.336,1098.00,1101.05,1097.67,1099.00,367.4268
2,binance,2022-11-10T00:10:30.798,1098.00,1101.05,1097.67,1098.51,450.0864
3,ftx,2022-11-10T00:10:35.308,1096.60,1098.20,1094.10,1097.80,8862.5352
4,ftx,2022-11-10T00:10:39.814,1096.60,1098.20,1094.10,1098.20,8862.5352
5,ftx,2022-11-10T00:10:44.889,1096.60,1098.20,1094.10,1098.20,8862.5352
6,bytetrade,2022-11-10T00:10:48.192,1096.05,1101.39,1096.05,1101.39,0.1833
7,bytetrade,2022-11-10T00:10:50.673,1096.05,1101.39,1096.05,1101.39,0.1833
8,bytetrade,2022-11-10T00:10:54.556,1096.05,1102.31,1096.05,1102.31,0.2233


<font color= lightblue> <font size = 2> Verification XRP </font>

In [68]:
v_params0= 25, criptos[2], exchanges[0]
v_params1= 25, criptos[2], exchanges[1]
v_params2= 25, criptos[2], exchanges[2]
v_params0, v_params1, v_params2

((25, 'XRP/USDT', 'binance'),
 (25, 'XRP/USDT', 'ftx'),
 (25, 'XRP/USDT', 'bytetrade'))

In [69]:
verif_0_X = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_X = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_X = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In this example, bigger spreads can be detected on bytetrade for XRP. On the contrary, binance has lower spreads and bigger volumes as well.

In [70]:
verif_df3 = fn.concatenate(verif_0_X[0], verif_1_X[0], verif_2_X[0], n)
verif_df3

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T00:10:57.689,0.335800,0.335700,1.579369e+06,908886.0000,0.000100,0.335700
1,binance,2022-11-10T00:11:00.240,0.335800,0.335700,1.587089e+06,911868.0000,0.000100,0.335700
2,binance,2022-11-10T00:11:02.991,0.335900,0.335800,1.579716e+06,829339.0000,0.000100,0.335800
3,ftx,2022-11-10T00:11:08.065,0.321350,0.320450,2.441700e+04,9655.0000,0.000900,0.320450
4,ftx,2022-11-10T00:11:13.016,0.321350,0.320450,2.441900e+04,18983.0000,0.000900,0.320450
5,ftx,2022-11-10T00:11:18.320,0.322275,0.320450,1.264140e+05,18986.0000,0.001825,0.321950
6,bytetrade,2022-11-10T00:11:21.669,0.350608,0.318693,1.211258e+02,2625.3629,0.031915,0.334713
7,bytetrade,2022-11-10T00:11:24.505,0.350608,0.318693,1.211258e+02,2625.3629,0.031915,0.334718
8,bytetrade,2022-11-10T00:11:27.890,0.350608,0.318693,1.211258e+02,2733.1040,0.031915,0.334509


<font color= lightblue> <font size = 2> OHLCV XRP </font>

In [71]:
verif_df3_OH = fn.concatenate(verif_0_X[1], verif_1_X[1], verif_2_X[1], n)
verif_df3_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T00:10:57.689,0.334600,0.335900,0.334400,0.335700,427367.000000
1,binance,2022-11-10T00:11:00.240,0.334600,0.335900,0.334400,0.335700,431677.000000
2,binance,2022-11-10T00:11:02.991,0.335800,0.335800,0.335800,0.335800,0.000000
3,ftx,2022-11-10T00:11:08.065,0.320425,0.320450,0.320425,0.320450,0.000000
4,ftx,2022-11-10T00:11:13.016,0.320425,0.321350,0.320425,0.320450,2.892150
5,ftx,2022-11-10T00:11:18.320,0.320425,0.321950,0.320425,0.321950,6.107975
6,bytetrade,2022-11-10T00:11:21.669,0.334506,0.334713,0.334439,0.334713,0.064500
7,bytetrade,2022-11-10T00:11:24.505,0.334506,0.334718,0.334439,0.334718,0.099600
8,bytetrade,2022-11-10T00:11:27.890,0.334506,0.334718,0.334439,0.334509,0.131000


<font color= orange> <font size = 5> 2. Microstructure Visualization </font>

Now we are going to obtain total_volumes, mid_prices and VWAP metric for criptocurrencies in every exchange.

In [72]:
help(vs.Micro)

Help on function Micro in module visualizations:

Micro(lvls, cripto, exchange, n, ts)
    Function that returns Microstructure OB and OHLCV for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable symbol (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    ts: timesleep (s) required in between data retrieval.
    
    Returns:
    -------
    Micro[0]: Order Books Data.
    Micro[1]: OHLCV Data.



<strong> 
<font color= lightgreen> <font size = 3> Progress check 2  </font> <strong>

Dataframes

<font color= lightblue> <font size = 2> Verification BTC </font>

In [73]:
params0= 25, criptos[0], exchanges[0]
params1= 25, criptos[0], exchanges[1]
params2= 25, criptos[0], exchanges[2]
params0,params1,params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

In [74]:
MS_0_B = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_B = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_B = vs.Micro(params2[0],params2[1], params2[2], n, .01)

Bid and Asks Volumes in the Microstructure determine the demand and it's a factor for price movements. This is also true for the VWAP which is a trend detector by reflecting the price adjusted by its volume.

In [75]:
MS_B = fn.concatenate(MS_0_B[0], MS_1_B[0], MS_2_B[0], n)
MS_B

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T00:11:31.104,25,1.06684,1.462160,2.529000,15997.23,22322.256299
1,binance,2022-11-10T00:11:33.161,25,1.29967,2.285880,3.585550,16000.17,20461.938687
2,binance,2022-11-10T00:11:34.713,25,4.87502,3.213040,8.088060,16009.23,17987.027194
3,ftx,2022-11-10T00:11:38.569,25,10.06690,9.468000,19.534900,15474.50,16249.093387
4,ftx,2022-11-10T00:11:40.600,25,5.95550,10.025500,15.981000,15482.00,16434.839184
5,ftx,2022-11-10T00:11:44.457,25,7.41760,9.442300,16.859900,15452.50,16361.027978
6,bytetrade,2022-11-10T00:11:46.582,25,3.50741,3.906414,7.413824,15995.65,18068.455605
7,bytetrade,2022-11-10T00:11:48.194,25,3.50741,3.906414,7.413824,15995.65,18068.455605
8,bytetrade,2022-11-10T00:11:51.172,25,3.50741,3.906414,7.413824,15995.65,18068.455605


<font color= lightblue> <font size = 2> Verification ETH </font>

In [76]:
params0= 25, criptos[1], exchanges[0]
params1= 25, criptos[1], exchanges[1]
params2= 25, criptos[1], exchanges[2]
params0,params1,params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

In [77]:
MS_0_E = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_E = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_E = vs.Micro(params2[0],params2[1], params2[2], n, .01)

In [78]:
MS_E = fn.concatenate(MS_0_E[0], MS_1_E[0], MS_2_E[0], n)
MS_E

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T00:11:53.232,25,37.5767,118.6440,156.2207,1105.370,1112.366207
1,binance,2022-11-10T00:11:54.976,25,58.5124,102.6719,161.1843,1106.690,1113.486426
2,binance,2022-11-10T00:11:56.489,25,36.6337,84.5239,121.1576,1107.485,1116.610987
3,ftx,2022-11-10T00:11:58.504,25,3.2720,10.7220,13.9940,1103.800,1181.840975
4,ftx,2022-11-10T00:12:00.370,25,0.1020,10.8150,10.9170,1103.800,1204.090803
5,ftx,2022-11-10T00:12:02.290,25,0.1020,10.8150,10.9170,1103.900,1204.290803
6,bytetrade,2022-11-10T00:12:04.492,24,19.0332,161.4905,180.5237,1107.290,1106.273594
7,bytetrade,2022-11-10T00:12:06.250,24,19.0332,161.4905,180.5237,1107.290,1106.273594
8,bytetrade,2022-11-10T00:12:08.043,25,19.1690,61.4963,80.6653,1108.020,1115.376102


<font color= lightblue> <font size = 2> Verification XRP </font>

In [79]:
params0= 25, criptos[2], exchanges[0]
params1= 25, criptos[2], exchanges[1]
params2= 25, criptos[2], exchanges[2]
params0,params1,params2

((25, 'XRP/USDT', 'binance'),
 (25, 'XRP/USDT', 'ftx'),
 (25, 'XRP/USDT', 'bytetrade'))

In [80]:
MS_0_X = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_X = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_X = vs.Micro(params2[0],params2[1], params2[2], n, .01)

In [81]:
MS_X = fn.concatenate(MS_0_X[0], MS_1_X[0], MS_2_X[0], n)
MS_X

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T00:12:10.052,25,1.632490e+06,598351.0000,2.230841e+06,0.337550,0.337500
1,binance,2022-11-10T00:12:11.604,25,1.603517e+06,593398.0000,2.196915e+06,0.337600,0.337500
2,binance,2022-11-10T00:12:13.320,25,1.486795e+06,574481.0000,2.061276e+06,0.337650,0.337600
3,ftx,2022-11-10T00:12:15.695,25,1.284190e+05,8481.0000,1.369000e+05,0.322462,0.321677
4,ftx,2022-11-10T00:12:20.358,25,1.274790e+05,17875.0000,1.453540e+05,0.322462,0.321677
5,ftx,2022-11-10T00:12:23.028,25,1.100080e+05,17732.0000,1.277400e+05,0.322462,0.321678
6,bytetrade,2022-11-10T00:12:26.124,25,1.570907e+02,2617.7242,2.774815e+03,0.337187,0.318821
7,bytetrade,2022-11-10T00:12:28.101,25,1.570907e+02,2617.7242,2.774815e+03,0.337187,0.318821
8,bytetrade,2022-11-10T00:12:29.960,25,1.212690e+02,2395.7231,2.516992e+03,0.337187,0.318834


<strong> 
<font color= lightgreen> <font size = 3> Progress check 2  </font> <strong>

Visualization

In [82]:
help(vs.Plot_line)

Help on function Plot_line in module visualizations:

Plot_line(data_ms, cripto, title)
    Function that plots Mid-Prices (y) and timestamps (x) for exchanges in data.
    Parameters:
    ----------
    data_ms: Microstructure data for column extraction ['exchanges', 'Mid_Prices'] and index timestamp as datetime.
    cripto: Downloadable cripto symbol.
    title: Title of facet_cols plotly lines before criptocurrency symbol (ex: 'Volume of': XRP/USDT)
    
    Returns:
    -------
    facet_col plots for Mid-Prices (col) during timestamps of data (index) for exchanges (col).



In [83]:
vs.Plot_line(MS_B, criptos[0], "Mid-Price of")

In [84]:
vs.Plot_line(MS_E, criptos[1], "Mid-Price of")

In [85]:
vs.Plot_line(MS_X, criptos[2], "Mid-Price of")

<font color= orange> <font size = 5> 3. Microstructure Modelling: </font>

### <font color= orange> <font size = 5> 4. Bibliography: </font>

<font color= lightgray>

+ Villalobos, O, 2021. **Python Project Template**. Available at: https://github.com/OmarVillalobos/python-project. 
(Accessed: November 5, 2022). 

+ Financial Engineering, **Marketmicrostructure.pptx**. ITESO -  Microstructure and Trading Systems (ITE1731 - DMAF). 
Available at: https://docs.google.com/presentation/d/1uYyMERKKGeZ0MPqbJ3bcir9lfpzbeAH9ZMmxbYysokQ/edit?usp=sharing (Accessed: November 5, 2022). 

+ Muñoz E., J **Roll Model** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/Roll%20.pdf (Accessed: November 5, 2022).

+ Muñoz E., J **Asset Pricing Theory** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/APT.pdf (Accessed: November 5, 2022). 